In [1]:
%%capture
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [2]:
%%capture
import os
import sys

!git clone https://github.com/14790897/sort-detectron2.git
os.chdir('/kaggle/working/sort-detectron2')
!pip install -r requirements.txt
sys.path.append('/kaggle/working/sort-detectron2')
os.chdir('..')

In [14]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from pathlib import Path

Data_Resister_training="coincide_separation_train";
Data_Resister_valid="coincide_separation_valid";
from detectron2.data.datasets import register_coco_instances
dataDir_train=Path('/kaggle/input/coco-data/images')
dataDir_test=Path('/kaggle/input/coco-data-val/images')

register_coco_instances(Data_Resister_training,{}, '/kaggle/input/coco-data/train.json', dataDir_train)
register_coco_instances(Data_Resister_valid,{},'/kaggle/input/coco-data/val.json', dataDir_train)

metadata = MetadataCatalog.get(Data_Resister_training)
dataset_train = DatasetCatalog.get(Data_Resister_training)
dataset_valid = DatasetCatalog.get(Data_Resister_valid)
print("类别名称: ", metadata.thing_classes)


[08/18 09:17:22 d2.data.datasets.coco]: Loaded 128 images in COCO format from /kaggle/input/coco-data/train.json
[08/18 09:17:22 d2.data.datasets.coco]: Loaded 33 images in COCO format from /kaggle/input/coco-data/val.json
类别名称:  ['coincide', 'separation']


In [21]:
import cv2,os
import torch
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import numpy as np
import matplotlib.pyplot as plt
from sort import *

# 初始化 SORT
sort_tracker = Sort()
track_history = {}  # 新增：用于存储每个track_id的历史轨迹

# 配置 Detectron2
cfg = get_cfg()
config_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(config_name))
cfg.DATASETS.TRAIN = (Data_Resister_training,)
cfg.DATASETS.TEST = (Data_Resister_valid,)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # 设置阈值
cfg.MODEL.WEIGHTS = (
    "/kaggle/input/detectron2_coincide_separation/pytorch/default/1/model_final.pth"
)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64  # 64 is slower but more accurate (128 faster but less accurate)
cfg.SOLVER.IMS_PER_BATCH = 2 #(2 is per defaults)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 # coincide and separation
cfg.INPUT.MASK_FORMAT='bitmask' #当指定bitmask的时候,只有rle格式可以使用，指定Polygon的时候只有Polygon格式可以使用
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.SOLVER.BASE_LR = 0.0005 #(quite high base learning rate but should drop)
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.WEIGHT_DECAY = 0.0005
cfg.SOLVER.GAMMA = 0.1

cfg.SOLVER.WARMUP_ITERS = 10 #How many iterations to go from 0 to reach base LR
cfg.SOLVER.MAX_ITER = 2000 #Maximum of iterations 1
cfg.SOLVER.STEPS = (500, 1000) #At which point to change the LR 0.25,0.5
cfg.TEST.EVAL_PERIOD = 250
cfg.SOLVER.CHECKPOINT_PERIOD=250
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

predictor = DefaultPredictor(cfg)

# 打开视频文件或相机
cap = cv2.VideoCapture("/kaggle/input/particle-video/output_video.mp4")

# 获取视频的宽度、高度和帧率
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 定义视频保存的格式和输出路径
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
out_video = cv2.VideoWriter(
    "output_video.avi",
    fourcc,
    fps,
    (frame_width, frame_height),
)
# 创建保存图像的文件夹
output_image_dir = "result_ini"
os.makedirs(output_image_dir, exist_ok=True)
# 打开 result_ini.txt 文件用于写入检测结果
with open(os.path.join(output_image_dir, "result_ini.txt"), "w") as result_file:
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 使用 Detectron2 进行目标检测
        outputs = predictor(frame)
        # 使用 Visualizer 绘制检测结果
        v = Visualizer(
            frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        #将绘制后的图像转换回 BGR 格式并写入视频
        frame = out.get_image()[:, :, ::-1]
        frame = np.array(frame)
        # 获取检测框
        boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()
        scores = outputs["instances"].scores.cpu().numpy()
        classes = outputs["instances"].pred_classes.cpu().numpy()
        # 获取类别名称
        class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes

        # 将框、分数和类别组合成 SORT 所需的输入格式
        detections = np.empty((0, 6))  # 包含类别信息的检测框格式
        for box, score, class_id in zip(boxes, scores, classes):
            detection = np.array([box[0], box[1], box[2], box[3], score, class_id])
            detections = np.vstack((detections, detection))

        # 更新 SORT 跟踪器
        tracks = sort_tracker.update(detections)

        # 绘制检测框和跟踪框
        for track in tracks:
            x1, y1, x2, y2, track_id, class_id = track[:6]
            if class_id>=0:
                class_name = class_names[int(class_id)]  # 获取类别名称
                print('int(class_id):',int(class_id))
                # 将检测结果写入 result_ini.txt
                result_file.write(
                    f"Frame {frame_count}: ID {int(track_id)}, class_id: {int(class_id)}, Class: {class_name}, Box [{x1}, {y1}, {x2}, {y2}]\n"
                )
                print(
                    "predict result:",
                    f"Frame {frame_count}: ID {int(track_id)}, Class: {class_name}, Box [{x1}, {y1}, {x2}, {y2}]\n",
                )
                # 在图像上绘制跟踪ID
                # cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {int(track_id)}", (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

                # 保存当前坐标到轨迹历史中
                track_id = int(track_id)
                if track_id not in track_history:
                    track_history[track_id] = []
                track_history[track_id].append((int(x1), int(y1), int(x2), int(y2)))
                # 绘制轨迹
                for i in range(1, len(track_history[track_id])):
                    pt1 = (track_history[track_id][i-1][0], track_history[track_id][i-1][1])
                    pt2 = (track_history[track_id][i][0], track_history[track_id][i][1])
                    cv2.line(frame, pt1, pt2, (0, 255, 0), 2)
            else:
                print('class id小于0，class_id',class_id)
        # 在这里添加写入视频文件的代码
        out_video.write(frame)
        # 保存当前帧为图像文件
#         frame_filename = os.path.join(output_image_dir, f"frame_{frame_count:04d}.jpg")
#         cv2.imwrite(frame_filename, frame)
        frame_count += 1
    cap.release()
    out_video.release()

from IPython.display import FileLink
FileLink(r'output_video.avi')

[08/18 09:26:20 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /kaggle/input/detectron2_coincide_separation/pytorch/default/1/model_final.pth ...
int(class_id): 0
predict result: Frame 0: ID 0, Class: coincide, Box [205.3253631591797, 55.76040267944336, 213.38815307617188, 63.45770263671875]

int(class_id): 0
predict result: Frame 0: ID 1, Class: coincide, Box [296.6557922363281, 573.275634765625, 303.1761474609375, 580.2236328125]

int(class_id): 0
predict result: Frame 0: ID 0, Class: coincide, Box [271.63275146484375, 571.0874633789062, 277.84625244140625, 577.228271484375]

int(class_id): 0
predict result: Frame 0: ID 1, Class: coincide, Box [355.59185791015625, 541.5277709960938, 363.7428283691406, 550.0439453125]

int(class_id): 0
predict result: Frame 0: ID 1, Class: coincide, Box [353.7958068847656, 609.1271362304688, 361.76190185546875, 616.9922485351562]

int(class_id): 0
predict result: Frame 0: ID 1, Class: coincide, Box [310.398681640625, 859.064

KeyboardInterrupt: 

In [ ]:
%%capture captured_output

# zip result_ini 
!zip -r result_ini.zip result_ini
# !rm -rf result_ini

In [ ]:
# # 设置图片文件夹和视频输出路径
# image_folder = 'result_ini'
# video_path = 'output_video.avi'

# # 获取所有图像文件，假设文件名形式为 "frame_1.jpg", "frame_2.jpg"...
# images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
# images.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # 根据数字部分排序

# # 读取一张图片以获得尺寸信息
# sample_img = cv2.imread(os.path.join(image_folder, images[0]))
# height, width, layers = sample_img.shape

# # 使用MJPG编解码器
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# video = cv2.VideoWriter(video_path, fourcc, 10.0, (width, height))  # 帧率设为20

# # 逐个添加图像到视频
# for image in images:
#     img = cv2.imread(os.path.join(image_folder, image))
#     video.write(img)

# # 释放VideoWriter
# video.release()
# from IPython.display import FileLink
# FileLink(r'output_video.avi')

In [ ]:
# fig, ax = plt.subplots(4, 1, figsize=(20, 50))
# indices = [ax[0], ax[1], ax[2], ax[3]]
# frame_count = 0
# for i in range(4):
#     indices[frame_count].imshow(out_frame)
#     indices[frame_count].grid(False)
#     frame_count += 1
# #     out_video.write(out_frame)

In [ ]:
from detectron2.utils.visualizer import ColorMode

import random
fig, ax = plt.subplots(4, 1, figsize =(20,50))
indices=[ax[0],ax[1],ax[2],ax[3] ]
# 列出目录中所有的文件
files = [os.path.join(output_image_dir, f) for f in os.listdir(output_image_dir) if os.path.isfile(os.path.join(output_image_dir, f))]

# 从文件列表中随机抽取4个文件
selected_files = random.sample(files, 4)

i = -1
for file_path in selected_files:
    i += 1    
    im = cv2.imread(file_path)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata, 
                   scale=1.0,
                   instance_mode=ColorMode.IMAGE_BW  # 需要从detectron2.utils.visualizer import ColorMode
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    processed_image = out.get_image()[:, :, ::-1]
    indices[i].imshow(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))  # 转换颜色空间以适配Matplotlib
    indices[i].grid(False)


In [13]:
# from PIL import Image
# from IPython.display import display

# # 读取图像
# img_path = '/kaggle/input/coco-data/images/1_C001H001S0002000020.jpg'
# img = Image.open(img_path)

# # 使用 IPython.display 来显示图像
# display(img)